# 🏭 ImgCraft Chapter Processor - Colab Runner
## Setup and Run
1. Chạy bước 1 để tải model và cài đặt môi trường.
2. Chạy bước 2 để khởi động Server và nhận public link.
Đừng quên Mount Google Drive để hệ thống đọc file từ app Local.

In [ ]:
# @title 1. Cài đặt Môi trường (FP8 Support) & Download Models
import os
import subprocess
from google.colab import drive

# 1. Kết nối Google Drive
drive.mount('/content/drive')

def run(cmd): 
    print(f"🖥️ Running: {cmd}")
    try:
        subprocess.run(cmd, shell=True, check=True)
    except subprocess.CalledProcessError as e:
        print(f"⚠️ Error on command: {cmd}")
        print(f"⚠️ Exit status: {e.returncode}")
        # Tiếp tục chạy dù có lỗi nhỏ (như swap)

# Bật SWAP để tránh OOM
try:
    print("💾 Tạo 16GB Swap File...")
    if not os.path.exists("/swapfile"):
        run("fallocate -l 16G /swapfile || dd if=/dev/zero of=/swapfile bs=1M count=16384")
        run("chmod 600 /swapfile")
        run("mkswap /swapfile")
    run("swapon /swapfile")
    print("✅ Swap Enabled.")
except Exception as e:
    print(f"⚠️ Swap cannot be enabled due to environment restrictions (ignoring): {e}")

# Cài dependencies
print("📦 System Dependencies...")
run("apt-get install -y aria2 libgl1-mesa-glx")

# Clone Repo Aniga Pipeline
os.chdir('/content')
if not os.path.exists('Aniga-Pipeline'):
    run("git clone https://github.com/jofix2004/Aniga-Pipeline.git")
else:
    os.chdir('Aniga-Pipeline')
    run("git fetch origin master")
    run("git reset --hard origin/master")
    os.chdir('/content')

# Clone ComfyUI (Dùng bản fork mod sẵn của jofix2004 thay vì bản gốc)
if not os.path.exists("/content/ComfyUI"):
    run("git clone https://github.com/jofix2004/ComfyUI.git")

print("📥 Python Requirements...")
run("pip uninstall -y comfy-cli comfy")
run("pip install -q -r /content/Aniga-Pipeline/aniga_imgcraft/requirements.txt")

print("🌐 Installing Cloudflare Tunnel...")
run("wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb")
run("dpkg -i cloudflared-linux-amd64.deb")

print("⬇️ Models...")
models = {
    "unet": [("flux1-kontext-dev-fp8-e4m3fn.safetensors", "https://huggingface.co/6chan/flux1-kontext-dev-fp8/resolve/main/flux1-kontext-dev-fp8-e4m3fn.safetensors")],
    "vae": [("ae.sft", "https://huggingface.co/Isi99999/Upscalers/resolve/main/Flux/ae.sft")],
    "clip": [("clip_l.safetensors", "https://huggingface.co/Isi99999/Upscalers/resolve/main/Flux/clip_l.safetensors"), 
             ("t5xxl_fp8_e4m3fn.safetensors", "https://huggingface.co/Isi99999/Upscalers/resolve/main/Flux/t5xxl_fp8_e4m3fn.safetensors")],
    "loras": [("flux_1_turbo_alpha.safetensors", "https://huggingface.co/Isi99999/Upscalers/resolve/main/Flux/flux_1_turbo_alpha.safetensors"),
              ("AniGaKontext2080v3_Full_000000650.safetensors", "https://huggingface.co/TranLinh2004/AniGaKontext2080v3_Full/resolve/main/AniGaKontext2080v3_Full_000000650.safetensors")]
}

for folder, items in models.items():
    dest = f"/content/ComfyUI/models/{folder}"
    os.makedirs(dest, exist_ok=True)
    for fname, url in items:
        if not os.path.exists(f"{dest}/{fname}"):
            run(f"aria2c -c -x 16 -s 16 -k 1M -d {dest} -o {fname} {url}")

print("✅ Môi trường & Models đã sẵn sàng!")


In [ ]:
# @title 2. Khởi động ImgCraft Server & Cloudflare Tunnel
import threading
import time
import subprocess
import sys
import os
import re
import uvicorn
import nest_asyncio

nest_asyncio.apply()

os.chdir('/content/Aniga-Pipeline/aniga_imgcraft')
sys.path.append('/content/Aniga-Pipeline/aniga_imgcraft')

import imgcraft_server
import imgcraft_core

print("📦 Khởi tạo FluxProcessor (Sử dụng Model GPU)...")
# Attach Flux Processor vào Server State ngay trong process hiện tại
imgcraft_server.state.flux_processor = imgcraft_core.FluxProcessor()

print("🚀 Bật ImgCraft Server (Port 8001)...")
server_config = uvicorn.Config(imgcraft_server.app, host="0.0.0.0", port=8001, log_level="error")
server = uvicorn.Server(server_config)
t = threading.Thread(target=server.run, daemon=True)
t.start()
time.sleep(3)

print("🌐 Bật Cloudflare Tunnel cho port 8001...")
cmd = "cloudflared tunnel --url http://localhost:8001"
process = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

print("="*50)
print("👇👇👇 LINK TRUY CẬP IMGCRAFT: 👇👇👇")
try:
    url_printed = False
    for line in iter(process.stdout.readline, ''):
        if not url_printed and "trycloudflare.com" in line:
            match = re.search(r"https://[a-zA-Z0-9-]+\.trycloudflare\.com", line)
            if match:
                print(f"\n🔗 URL: {match.group(0)}\n")
                print("="*50)
                url_printed = True
        # Giữ vòng lặp để giữ cell chay mãi mãi, không bị end process
except KeyboardInterrupt:
    process.kill()
    print("\n🛑 Tunnel stopped.")
